### This notebook showcases how to use a HyDE Rewrite IO Processer

* HyDERewriteIOProcessor only considers the last user input
* It then asks the backend to generate an answer for the user input
* Finally, it enriches the last user input by appending the generated answer

The modified user input in essence repreresents "Query + Answer". This could lead to better Retrieval

In [ ]:
from granite_io.backend.vllm_server import LocalVLLMServer
from granite_io.io.granite_3_3.input_processors.granite_3_3_input_processor import (
    Granite3Point3Inputs,
)

In [ ]:
from granite_io.io.hyde_rewrite import HyDERewriteIOProcessor

In [ ]:
model_name = "ibm-granite/granite-3.3-8b-instruct"

In [ ]:
server = LocalVLLMServer(model_name)

In [ ]:
server.wait_for_startup(200)

In [ ]:
backend = server.make_backend()

In [ ]:
# Create an example chat completion with a short conversation.
chat_input = Granite3Point3Inputs.model_validate(
    {
        "messages": [
            {"role": "assistant", "content": "Welcome to pet questions!"},
            {
                "role": "user",
                "content": "I have two pets, a dog named Rex and a cat named Lucy.",
            },
            {
                "role": "assistant",
                "content": "Great, what would you like to share about them?",
            },
            {
                "role": "user",
                "content": "Rex spends a lot of time in the backyard and outdoors, "
                "and Luna is always inside.",
            },
            {
                "role": "assistant",
                "content": "Sounds good! Rex must love exploring outside, while Lucy "
                "probably enjoys her cozy indoor life.",
            },
            {
                "role": "user",
                "content": "But is he more likely to get fleas because of that?",
            },
        ],
        "generate_inputs": {"temperature": 0.0},
    }
)
chat_input

In [ ]:
io_proc = HyDERewriteIOProcessor(backend)

In [ ]:
chat_result = await io_proc.acreate_chat_completion(chat_input)
print(chat_result.results[0].next_message.model_dump_json(indent=2))

In [ ]:
chat_result